# Lab work journal

Notes for each sessions of lab work.

### 2018-04-10 Genome assembly
Made soft links to the data located in the /ga2018/data folders.
Started writing bash script for the batch assembling the genome, will have to wait for next lab session.
Want to test running megahit in an interactive session first to see that it works. 

### 2018-04-12 UPPMAX down
Uppmax down for maintenance. Working on preparation of the batch script so that it can be submitted as soon as UPPMAX comes online again.